In [1]:
import pandas as pd
import numpy as np
import string, os
import warnings
warnings.filterwarnings("ignore")
warnings.simplefilter(action='ignore', category=FutureWarning)

In [2]:
# set seeds for reproducability
import tensorflow
from numpy.random import seed
tensorflow.random.set_seed(2)
seed(1)



# keras module for building LSTM 
from keras_preprocessing.sequence import pad_sequences
from keras.layers import Embedding, LSTM, Dense, Dropout
from keras.preprocessing.text import Tokenizer
from keras.callbacks import EarlyStopping
from keras.models import Sequential
import keras.utils as ku

In [4]:
# Loading the all headlines as a list 
curr_dir = 'C:\\Users\\sarathbabu.karunanit\\Downloads\\Tutorials\\Transformers\\archive\\'
all_headlines = []
for filename in os.listdir(curr_dir):
    if 'Articles' in filename:
        print(filename)
        article_df = pd.read_csv(curr_dir+filename)
        all_headlines.extend(list(article_df.headline.values))
        #break

ArticlesApril2017.csv
ArticlesApril2018.csv
ArticlesFeb2017.csv
ArticlesFeb2018.csv
ArticlesJan2017.csv
ArticlesJan2018.csv
ArticlesMarch2017.csv
ArticlesMarch2018.csv
ArticlesMay2017.csv


In [5]:
all_headlines = [line for line in all_headlines if line!= "Unknown"]
print(all_headlines[:10])

['Finding an Expansive View  of a Forgotten People in Niger', 'And Now,  the Dreaded Trump Curse', 'Venezuela’s Descent Into Dictatorship', 'Stain Permeates Basketball Blue Blood', 'Taking Things for Granted', 'The Caged Beast Awakens', 'An Ever-Unfolding Story', 'O’Reilly Thrives as Settlements Add Up', 'Mouse Infestation', 'Divide in G.O.P. Now Threatens Trump Tax Plan']


In [6]:
len(all_headlines)

8603

In [7]:
def clean_text(txt):
    txt = "".join(t for t in txt if t not in string.punctuation).lower()
    txt = txt.encode("utf8").decode("ascii",'ignore')
    return txt

In [8]:
corpus = [clean_text(x) for x in all_headlines]
print(corpus[:10])

['finding an expansive view  of a forgotten people in niger', 'and now  the dreaded trump curse', 'venezuelas descent into dictatorship', 'stain permeates basketball blue blood', 'taking things for granted', 'the caged beast awakens', 'an everunfolding story', 'oreilly thrives as settlements add up', 'mouse infestation', 'divide in gop now threatens trump tax plan']


In [26]:
tokenizer = Tokenizer()
def get_sequence_of_tokens(corpus):
    ## tokenization
    tokenizer.fit_on_texts(corpus)
    total_words = len(tokenizer.word_index) + 1
    
    ## convert data to a token sequence 
    input_sequences = []
    for line in corpus:
        token_list = tokenizer.texts_to_sequences([line])[0]
        for i in range(1, len(token_list)):
            n_gram_sequence = token_list[:i+1]
            input_sequences.append(n_gram_sequence)
    return input_sequences, total_words

In [45]:
total_words

11265

In [27]:
inp_sequences, total_words = get_sequence_of_tokens(corpus)
print(inp_sequences[:10])

[[391, 17], [391, 17, 5166], [391, 17, 5166, 523], [391, 17, 5166, 523, 4], [391, 17, 5166, 523, 4, 2], [391, 17, 5166, 523, 4, 2, 1601], [391, 17, 5166, 523, 4, 2, 1601, 134], [391, 17, 5166, 523, 4, 2, 1601, 134, 5], [391, 17, 5166, 523, 4, 2, 1601, 134, 5, 1951], [7, 57]]


In [36]:
def generate_padded_sequences(input_sequences):
    max_sequence_len = max([len(x) for x in input_sequences])
    input_sequences = np.array(pad_sequences(input_sequences, maxlen=max_sequence_len, padding='pre'))
    
    predictors, label = input_sequences[:,:-1],input_sequences[:,-1]
    label = ku.to_categorical(label, num_classes=total_words)
    return predictors, label, max_sequence_len

In [37]:
predictors, label, max_sequence_len = generate_padded_sequences(inp_sequences)

In [59]:
predictors.shape

(51770, 23)

In [60]:
label.shape

(51770, 11265)

In [44]:
max_sequence_len

24

In [46]:
def create_model(max_sequence_len, total_words):
    input_len = max_sequence_len - 1
    model = Sequential()
    # ----------Add Input Embedding Layer
    model.add(Embedding(total_words, 10, input_length=input_len))
    # ----------Add Hidden Layer 1 - LSTM Layer
    model.add(LSTM(100))
    model.add(Dropout(0.1))
    # ----------Add Output Layer
    model.add(Dense(total_words, activation='softmax'))
    model.compile(loss='categorical_crossentropy', optimizer='adam')
    return model

In [47]:
model = create_model(max_sequence_len, total_words)
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 23, 10)            112650    
                                                                 
 lstm (LSTM)                 (None, 100)               44400     
                                                                 
 dropout (Dropout)           (None, 100)               0         
                                                                 
 dense (Dense)               (None, 11265)             1137765   
                                                                 
Total params: 1,294,815
Trainable params: 1,294,815
Non-trainable params: 0
_________________________________________________________________


In [49]:
model.fit(predictors, label, epochs=10)

Epoch 1/10
1618/1618 [==============================] - 41s 25ms/step - loss: 7.7474
Epoch 2/10
1618/1618 [==============================] - 42s 26ms/step - loss: 7.4271
Epoch 3/10
1618/1618 [==============================] - 43s 26ms/step - loss: 7.2304
Epoch 4/10
1618/1618 [==============================] - 47s 29ms/step - loss: 7.0260
Epoch 5/10
1618/1618 [==============================] - 49s 30ms/step - loss: 6.8018
Epoch 6/10
1618/1618 [==============================] - 52s 32ms/step - loss: 6.5541
Epoch 7/10
1618/1618 [==============================] - 52s 32ms/step - loss: 6.3036
Epoch 8/10
1618/1618 [==============================] - 55s 34ms/step - loss: 6.0638
Epoch 9/10
1618/1618 [==============================] - 53s 33ms/step - loss: 5.8318
Epoch 10/10
1618/1618 [==============================] - 54s 34ms/step - loss: 5.6068


In [52]:
def generate_text(seed_text, next_words, model, max_sequence_len):
    for _ in range(next_words):
        token_list = tokenizer.texts_to_sequences([seed_text])[0]
        token_list = pad_sequences([token_list],              maxlen=max_sequence_len-1, padding='pre')
        #predicted = model.predict_classes(token_list, verbose=0)
        
        predict_x=model.predict(token_list,verbose=0) 
        predicted=np.argmax(predict_x,axis=1)
        
        output_word = ""
        for word,index in tokenizer.word_index.items():
            if index == predicted:
                output_word = word
                break
        seed_text += " "+output_word
    return seed_text.title()

In [58]:
print (generate_text("india and pakistan", 10, model, max_sequence_len))
print (generate_text("president trump", 10, model, max_sequence_len))
print (generate_text("united states", 10, model, max_sequence_len))
print (generate_text("donald trump", 10, model, max_sequence_len))
print (generate_text("new york", 10, model, max_sequence_len))
print (generate_text("science and technology", 10, model, max_sequence_len))

India And Pakistan On The World Of A New Yorker Times For The
President Trump Is A New York Of Discipline And A New Yorker
United States Live To A New York Of Trees And A New
Donald Trump And The World Of A New Yorker Times For The
New York Today A Noreaster Downer And A Trifle For The New
Science And Technology A New York Of Discipline And A New Yorker With
